**Amazon Nova Text Understanding notebook**

**Introduction**

Welcome to the Amazon Nova Text Understanding notebook! We'll explore the powerful capabilities of Amazon Nova to ingest charts and text and generate insights. 

**Use Case**

This use case demonstrates Nova's ability to summarize complex data from charts/ graphs. You are Bob, an Analyst at AnyCompany and you need to explain to your leaders last 5 years trend in cost reduction across various supplier categories as depicted in a chart. You need the help of a Foundation Model to summarize all insights from this chart.

**Workshop Objectives**

By the end of this workshop, you will:

1. Understand the key features of Amazon Nova
2. Learn how to use these features for a real-world scenario
3. Gain hands-on experience with the Amazon Bedrock API 



In [ ]:
# Install and read in required packages
%pip install boto3 IPython pdf2image

In [ ]:
import base64
import boto3
import json
from IPython.display import Image

modelId = "us.amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"
bedrock_rt = boto3.client("bedrock-runtime", region_name="us-east-1") 

In [ ]:

with open("./imgs/cvna_2021_annual_report_image.png", "rb") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")

In [ ]:
def get_completion(messages):
    prompt_config = {
        "schemaVersion": "messages-v1",
        "inferenceConfig": {
            "max_new_tokens": 300,
            "top_p": 0.9,
            "top_k": 20,
            "temperature": 0.7,
        },
        "messages": messages
    }
    body = json.dumps(prompt_config)
    response = bedrock_rt.invoke_model(
        body=body, modelId=modelId
    )
    response_body = json.loads(response["body"].read())
    results = response_body["output"]["message"]["content"][0]["text"]
    return results

Let's see how we can pass this image to the model alongside a simple question.

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {
                "image": {
                    "format": "png",
                    "source": {"bytes": base64_string},
                }
            },        
            {
                "text": "What's in this image? Answer in a single sentence."
            }
        ]
    }
]

get_completion(messages)


The image shows various financial metrics and growth trends from Carvana's 2021 annual report, including retail units sold, total revenue, total markets, and car vending machines over the years 2014 to 2021.
That's pretty good! Now let's ask it some more useful questions.

In [ ]:
questions = [
    "What was CVNA revenue in 2020?",
    "How many additional markets has Carvana added since 2014?",
    "What was 2016 revenue per retail unit sold?"
]

for index, question in enumerate(questions):
    messages = [
        {
            
            "role": 'user',
            "content": [
                {
                    "image": {
                        "format": "png",
                        "source": {"bytes": base64_string},
                    }
                },
                {
                    "text": question
                }
            ]
        }
    ]

    print(f"\n----------Question {index+1}----------")
    print(get_completion(messages))

As you can see, Nova is capable of answering fairly detailed questions about charts and graphs. 

**Summary**

In this notebook, we explored the powerful features of Amazon Nova through the lens of Carvana. 

You have now experimented with using boto3 SDK to get an exposure to Amazon Nova and Amazon Bedrock API. Using this API you have analyzed different images and derived meaningful insights from the image.